In [ ]:
# To run this notebook please follow the instructions given in ReadMe file.

# This notebook contains the code to train the model and save it.

In [ ]:
#Mount Google drive for importing dataset, saved model
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#opencv v4.5.4.58 is required for smooth running of the code.

!pip install --upgrade opencv-python

In [ ]:
# Import all necessary modules.
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
import cv2
import os
import pickle 

In [ ]:
# Dataset Location
images_loc='/content/drive/MyDrive/Unbiased_Enthusiasts/Our_model/Dataset/Train_data'  

#Labels location
label_loc='/content/drive/MyDrive/Unbiased_Enthusiasts/Our_model/Dataset/train.csv'  

In [ ]:
#Reading the lable data
labels=pd.read_csv(label_loc) 

In [ ]:
# names - list of names of images
# count_df - list of number of people in the image

names=[] 
count_df=[]  
for i in range(len(labels)):
  names.append(labels['Name'][i])
  count_df.append(labels['HeadCount'][i])

In [ ]:
# This is size of the training set. Increase it for training on larger set.
n = 2500

def load_images_from_folder(folder,names,cou):

    """ this function takes the images(Total=n) and resize them to (256,256)
        resolution and prepares the data for training """

    images = []
    y=[]
    c=0
    for filename in os.listdir(folder):
      if filename in names:
        index=names.index(filename)
        c=c+1

        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.resize(img,(256,256))
        if img is not None:
            images.append(img)
            y.append(cou[index])
        if c==n:
          break
    return [images,y]

In [ ]:
# Load data for training
train_data=load_images_from_folder(images_loc,names,count_df) 

In [ ]:
# Changing the label data into an array
y=train_data[1][0:n]
y=np.array(y)
y.shape

(2500,)

In [ ]:
# Changing the dataset into an array
img2=np.array(train_data[0])
img2.shape

(2500, 256, 256, 3)

In [ ]:
# setting features and target value
x_train, x_val, y_train, y_val = train_test_split(img2, y, test_size=0.1)
print(x_train.shape[0])
print(x_val.shape[0])

2250
250


In [ ]:
# Create the model
# The model is a Convolutional neural network.
# The details regarding its architecture are printed below the cell.

model = tf.keras.Sequential([
    
    tf.keras.layers.Conv2D(64, (3,3), input_shape=(256,256,3), activation=tf.keras.activations.relu),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation=tf.keras.activations.relu),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(7),
    tf.keras.layers.Dense(1)
    
])

model.compile(loss=tf.keras.losses.Huber(), optimizer=tf.keras.optimizers.Adam(), metrics=['mae'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 254, 254, 64)      1792      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 127, 127, 64)     0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 125, 125, 128)     73856     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 62, 62, 128)      0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 62, 62, 128)       0         
                                                                 
 flatten_1 (Flatten)         (None, 492032)           

In [ ]:
# add a learning rate monitor to get the lr with smoothest prediction

lr_monitor = tf.keras.callbacks.LearningRateScheduler(
                lambda epochs : 1e-8 * 10 ** (epochs/20))


In [ ]:
# train the model 
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, batch_size=16, callbacks=[lr_monitor])

Epoch 1/50
141/141 [==============================] - 23s 159ms/step - loss: 33.9086 - mae: 34.4075 - val_loss: 10.5593 - val_mae: 11.0489 - lr: 1.0000e-08
Epoch 2/50
141/141 [==============================] - 22s 158ms/step - loss: 15.7963 - mae: 16.2888 - val_loss: 9.1403 - val_mae: 9.6321 - lr: 1.1220e-08
Epoch 3/50
141/141 [==============================] - 22s 158ms/step - loss: 15.0275 - mae: 15.5218 - val_loss: 9.2580 - val_mae: 9.7475 - lr: 1.2589e-08
Epoch 4/50
141/141 [==============================] - 22s 158ms/step - loss: 15.6506 - mae: 16.1420 - val_loss: 8.7129 - val_mae: 9.2044 - lr: 1.4125e-08
Epoch 5/50
141/141 [==============================] - 22s 157ms/step - loss: 14.7310 - mae: 15.2233 - val_loss: 9.0460 - val_mae: 9.5346 - lr: 1.5849e-08
Epoch 6/50
141/141 [==============================] - 22s 157ms/step - loss: 14.6327 - mae: 15.1257 - val_loss: 8.3164 - val_mae: 8.8019 - lr: 1.7783e-08
Epoch 7/50
141/141 [==============================] - 22s 157ms/step - los

In [ ]:
# Save the trained model for future use.

Pkl_Filename = "Pickle_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)

INFO:tensorflow:Assets written to: ram://b15d68cb-b905-455b-b1a7-973f884d4a21/assets


In [ ]:
# Save the file to drive
!cp Pickle_Model.pkl /content/drive/MyDrive/Unbiased_Enthusiasts/Our_model/Saved_model